# Operaciones que realiza un mecanismo de atención

Primero se importan las librerías en el cabecero

In [54]:
import numpy as np
import scipy.special as sci
import pandas as pd

El corpus en este caso es inicializado de forma aleatoria y representa a los embeddings ya con codificación posicional.
En la práctica se pueden tomar estos embeddings de una librería como word2vec.

In [55]:
# np.random.seed(42)  # Valor aleatorio de seed, el cual permite poder replicar los resultados
corpus = np.around(np.random.rand(3,3),3)
corpus.shape
print(f"Corpus shape: {corpus.shape}\n\n")
print(f"Corpus:\n{corpus}")

Corpus shape: (3, 3)


Corpus:
[[0.285 0.037 0.61 ]
 [0.503 0.051 0.279]
 [0.908 0.24  0.145]]


A partir de los embeddings iniciales se obtienen 3 matrices, Q, K y V, donde cada una es una copia de dichos embeddings.

In [56]:
Q = corpus.copy()
K = corpus.copy()
V = corpus.copy()

print(f"Q shape: {Q.shape}\n\n")
print(f"Q:\n{Q}")

Q shape: (3, 3)


Q:
[[0.285 0.037 0.61 ]
 [0.503 0.051 0.279]
 [0.908 0.24  0.145]]


En este caso la matriz de pesos inicia de forma aleatoria, en la práctica estas matrices pueden ya estar dadas gracias a entrenamientos previos para ahorrar tiempo y entrenamiento o bien inicializarse de forma aleatoria.

In [57]:
W_Q = np.around(np.random.rand(3,3),3)
W_K = np.around(np.random.rand(3,3),3)
W_V = np.around(np.random.rand(3,3),3)

print(f"Key weights: {W_K}\n\n")
print(f"Query weights: {W_Q}\n\n")
print(f"Values weights: {W_V}\n\n")

Key weights: [[0.634 0.536 0.09 ]
 [0.835 0.321 0.187]
 [0.041 0.591 0.678]]


Query weights: [[0.489 0.986 0.242]
 [0.672 0.762 0.238]
 [0.728 0.368 0.632]]


Values weights: [[0.017 0.512 0.226]
 [0.645 0.174 0.691]
 [0.387 0.937 0.138]]




Se multiplica cada matriz (Q,K,V) con su respectiva matriz de pesos.

Dicha matriz de pesos puede ser incializada (y en este caso es así) de forma aleatoria, sin embargo, estos valores se pueden tener pre-entrenados, de tal manera que ahorramos el ajuste de estos pesos.

In [58]:
Q = np.dot(Q,W_Q)
K = np.dot(K,W_K)
V = np.dot(V,W_V)
Q_original = corpus.copy()

print(f"Key (Post adding weights): \n{K}\n\n")
print(f"Query (Post adding weights): \n{Q}\n\n")
print(f"Values (Post adding weights): \n{V}\n\n")

Key (Post adding weights): 
[[0.236595 0.525147 0.446149]
 [0.372926 0.450868 0.243969]
 [0.782017 0.649423 0.22491 ]]


Query (Post adding weights): 
[[0.608309 0.533684 0.463296]
 [0.483351 0.637492 0.310192]
 [0.710852 1.131528 0.368496]]


Values (Post adding weights): 
[[0.26478  0.723928 0.174157]
 [0.149419 0.527833 0.187421]
 [0.226351 0.642521 0.391058]]




Se tiene que realizar la siguiente operación: 
$$\text{Attention(Q,K,V)}=\text{softmax}(\frac{QK^T}{\sqrt{d_k}})$$

$\text{res}=QK^T$

In [59]:
res = np.dot(Q,K.T)
res

array([[0.63088447, 0.58050514, 0.92649455],
       [0.58752729, 0.54335613, 0.86175595],
       [0.92680669, 0.86516656, 1.37361709]])

$\text{res}=\frac{\text{res}}{\sqrt{d_k}}=\frac{QK^T}{\sqrt{d_k}}$

In [60]:
res = res/np.sqrt(K.shape[0]+K.shape[1])
res

array([[0.2575575 , 0.23699023, 0.37823981],
       [0.23985701, 0.22182421, 0.35181039],
       [0.37836724, 0.35320277, 0.56077683]])

$\text{res}=\text{softmax}(\text{res})=\text{softmax}(\frac{QK^T}{\sqrt{d_k}})$

In [61]:
res = sci.softmax(res,axis=0)
print(f"Sum = {np.sum(res,axis=0)}")
# e_x = np.exp(res[:1] - np.max(res))
# res = e_x / e_x.sum()
# print(f"Sum = {np.sum(res)}")

Sum = [1. 1. 1.]


# Multiplicación con los valores originales
Esto se realiza para agregar la información que ya está contextualizada con la información original.

Esto se hace con el objetivo de que la matriz original contenga ya la información contextual, es decir, los tokens (en este caso palabras) ya conocen su relación con el resto de palabras del corpus.

In [62]:
res = res*V
res

array([[0.08511458, 0.23291035, 0.05486685],
       [0.04718863, 0.16726437, 0.05750547],
       [0.08210473, 0.23219412, 0.1478717 ]])

Se aplica un redondeo con el objetivo de hacer más fácil la visualización

In [63]:
res = np.around(res,4)
res

array([[0.0851, 0.2329, 0.0549],
       [0.0472, 0.1673, 0.0575],
       [0.0821, 0.2322, 0.1479]])

# Capa de normalización

Se puede ver en el diagrama de la arquitectura, que posterior a la capa de atención, hay una etapa denominada "Add & Norm" de color amarillo.

Esto indica que suma el resultado obtenido con la matriz N (número de tokens) $\times$ M (longitud del embedding) original y posteriormente se realiza una normalización. Esta normalización comúnmente es la denominada `layer_normalization`.

Este proceso se muestra en las siguientes celdas

In [64]:
def layer_norm(x, epsilon=1e-6):
    mean = np.mean(x, axis=1, keepdims=True)  # Mean across features
    variance = np.var(x, axis=1, keepdims=True)  # Variance across features
    normalized = (x - mean) / np.sqrt(variance + epsilon)  # Normalization
    return normalized

In [65]:
res = layer_norm(res+Q_original)

# Representación visual

Se convierte en DataFrame con el fin de poder visualizar que cada embedding es una representación vectorial de los tokens y que el mecanismo de atención tiene como fin entender la relación que tienen cada palabra con las otras del corpus.

Se puede ver que en la matriz, el valor que se encuentre en la coordenada (renglón, columna) será la representación de la relación entre tokens, en donde un valor más grande indica una relación más fuerte entre palabras.

In [66]:
res_df = pd.DataFrame(res)
res_df.columns=["Amo","el","queso"]
res_df.index=["Amo","el","queso"]
res_df

,Amo,el,queso
Amo,-0.386900,-0.984547,1.371447
el,1.324032,-1.092278,-0.231754
queso,1.370163,-0.381810,-0.988353
